In [ ]:
import os
exec(open("Libs_Modulus.py").read())
exec(open("Links.py").read())
exec(open("ConvertTool.py").read())
"""exec(open("CheckDirFiles.py").read())"""

def CHECK_DATA_DIR(MainDir,DataType):
    data_dir = os.path.join(MainDir, DataType)
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        ExistDir = False
    else :
        ExistDir = True
    print("Le répertoire '", DataType, "' exist : ", np.where(ExistDir==True, "OK", "NOK \nCréation du répertoire"))
    return ExistDir

# Mise à jour des fichiers Excel téléchargés pour les rendre exploitables
def UPDATE_FORMAT(file_name, name_filter, change):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)           
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)

# Obtention de la liste des fichiers de classements journaliers
def GET_RANK_LIST (RankURL):
    soup = BeautifulSoup(requests.get(RankURL).content.decode("utf-8"))
    ListSoup = soup.findAll('option')
    RankList = []
    for iRankList in ListSoup:
        RankList.append(iRankList.text[2:])
    RankList = np.unique(RankList[1:-1])
    print ("Il y a ", len(RankList), "fichiers Excel de classement")
    return RankList
        
# Téléchargement de fichier de classement journalier
def DWNLD_DAILY_RANK_FILE(DLRankURL, TFN, RankFileType, MainDir, DataDir):
    os.chdir(MainDir+DataDir)
    
    with tqdm(range(len(TFN)), desc = "Chargement des données Excel") as outer:
        for iTFN in TFN:
            resp = requests.get(DLRankURL+iTFN+RankFileType)
            print("vendeeglobe_"+iTFN+RankFileType)
            with open("vendeeglobe_"+iTFN+RankFileType,'wb') as file:
                file.write(resp.content)
            UPDATE_FORMAT("vendeeglobe_"+iTFN+RankFileType, name_filter='xl/styles.xml', change=lambda d: re.sub(b'xxid="\d*"', b"", d))
            outer.update()
    os.chdir(MainDir)

# Vérification de l'existance de tous les fichiers à télécharger
def CHECK_FILE_LIST(MainDir, DataRankDir, TFN):
    Directory = os.path.join(MainDir, DataRankDir)
    FilesList = [f for f in listdir(Directory) if isfile(join(Directory, f))]
    #print(FilesList)
    ListToDownLoad = []
    for iTFN in TFN:
        if  not str("vendeeglobe_"+iTFN+".xlsx") in FilesList:
            ListToDownLoad.append(iTFN)
            pass
    return ListToDownLoad


""" MAIN """
ExistDir = CHECK_DATA_DIR(MainDir,DataRankDir)
RankList = GET_RANK_LIST (RankURL)
TL,TFN = CONVERT_TIME_FORMAT(RankList)
if ExistDir == False :
    DWNLD_DAILY_RANK_FILE(DLRankURL, TFN, RankFileType, MainDir, DataRankDir)
else :
    ListToDownLoad = CHECK_FILE_LIST(MainDir, DataRankDir, TFN)
    if len(ListToDownLoad)!=0:
        DWNLD_DAILY_RANK_FILE(DLRankURL, ListToDownLoad, RankFileType, MainDir, DataRankDir)


In [ ]:
def GEN_SEPCS_DF (SkippersURL):
    # Paramètres des types de données à extraire
    DataParam = {"BoatSpecs" : ['ul', "class", "boats-list__popup-specs-list"],
                 "Nom voilier" : ['h3',"class","boats-list__boat-name"],
                 "Nom skipper" : ["span","class", "boats-list__skipper-name"]}
    # Fonction d'extraction de données techniques spécifiques (BoatSpecs, BoatName, SkipperName) sous forme de list
    LIST_SOUP = lambda SkippersURL, DataParam,ikey : BeautifulSoup(requests.get(SkippersURL).content.decode("utf-8"))\
    .findAll(DataParam[ikey][0],
             {DataParam[ikey][1]:
              DataParam[ikey][2]})
    
    # Création d'un DF Spécifications techniques du voilier vierge
    SpecsDF=pd.DataFrame()
    
    # Remplissage du DF
    for ikey in DataParam:
        ListSoup = LIST_SOUP (SkippersURL, DataParam, ikey)
        
        # Remplir d'abord les données techniques du voilier (Num. voile, Anciens nom, Architece, Chantier, Date lanc., Longueur,...)
        if ikey == list(DataParam.keys())[0]:
            i = 0
            for iul in ListSoup:
                idata,ivalue = np.transpose([string.split(' : ',1) for string in iul.text.split("\n")[1:-1]])
                SpecsDF = pd.concat([SpecsDF,pd.DataFrame([ivalue],columns = idata)],ignore_index=True)
                i+=1
        # Ajouter ensuite les colonnes Nom de voilier, et nom du marin; puis les mettre en premières colonnes
        else:
            SpecsDF[ikey] = [string.text for string in ListSoup]
            SpecsDF = SpecsDF[SpecsDF.columns.tolist()[-1:]+SpecsDF.columns.tolist()[:-1]]
    
    # Homogénéisation des données
       # liste des valeurs [init, new] et des unités à supprimer suivant le nom de la colonne 
    ValToReplace = {"Date de lancement":[], # pour reconvertion des dates
                    "Anciens noms du bateau":[np.nan,''],"Voile quille":[np.nan,''], # Pour remplacement des valeurs
                    "Nombre de dérives":[["foiler", "2 asymétriques"],['foils', "2"]], # Pour remplacement des valeurs
                    "Longueur":[' m'],"Largeur" :[' m'],"Tirant d'eau":[' m'],"Hauteur mât":[' m'], # Suppresion des unités de longueur
                    "Surface de voiles au près":[' m2', ' m²'],"Surface de voiles au portant":[' m2', ' m²'], #Suppression des unités de surface
                    "Déplacement (poids)":[' t', ' tonnes']} #Suppresion des unités de masse
    for icol in ValToReplace:
        if icol in ["Anciens noms du bateau","Voile quille","Nombre de dérives"]:      
            SpecsDF[icol] = SpecsDF[icol].replace(ValToReplace[icol][0],ValToReplace[icol][1])
        elif "Date de lancement" in icol:
            SpecsDF[icol] = SpecsDF[icol].apply(dateparser.parse)
        else :
            for iunit in ValToReplace[icol]:
                SpecsDF[icol] = SpecsDF[icol].str.strip(iunit)
            if "Déplacement (poids)" in icol:
                SpecsDF[icol] = SpecsDF[icol].str.replace('[a-zA-Z]', '0', regex=True)
            SpecsDF[icol] = SpecsDF[icol].str.replace(',','.')
            SpecsDF[icol] = SpecsDF[icol].astype('float')
    return SpecsDF

In [ ]:
Specs_DF = GEN_SEPCS_DF(SkippersURL)
Specs_DF.head()

In [ ]:
XLSX_DIR = os.path.join(MainDir, DataRankDir)
os.chdir(XLSX_DIR)
df = pd.read_excel ("vendeeglobe_20210305_080000.xlsx", header=3)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df["Date"]='2021/03/05 08:00:00'
print('Attention! Date en format str à convertir en format date')

df["Date d'arrivée\nArrival date"] = df["Unnamed: 7"]
df.loc[1:,"Écarts\nGaps"] = df.loc[1:,"Unnamed: 13"]
df.loc[1:,"Unnamed: 14"] = df.loc[1:,"Unnamed: 15"]
df = df.drop(["Unnamed: 5","Unnamed: 6","Unnamed: 7", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 13", "Unnamed: 15"], 1)
df.head()

In [ ]:
os.chdir(XLSX_DIR)
print(XLSX_DIR)
df = pd.read_excel ("vendeeglobe_20201108_140000.xlsx", header=3)

In [ ]:
os.chdir(XLSX_DIR)
print(XLSX_DIR)
df = pd.read_excel ("vendeeglobe_20201109_080000.xlsx", header=3)

In [ ]:
TFN[4]

In [ ]:
pd.read_excel ("vendeeglobe_"+TFN[0]+".xlsx", header=3)

In [ ]:
# print(TFN)
for i in TFN:
#     print (i)
    UPDATE_FORMAT("vendeeglobe_"+i+RankFileType, name_filter='xl/styles.xml', change=lambda d: re.sub(b'xxid="\d*"', b"", d))

    df = pd.read_excel ("vendeeglobe_"+i+".xlsx", header=3)

In [ ]:
os.chdir(XLSX_DIR)
print(XLSX_DIR)
df = pd.read_excel ("vendeeglobe_20201201_040000.xlsx", header=3)
"""df = df.rename(columns={'Unnamed: 0': 'Date'})
df["Date"]='2021/03/05 14:00:00'
print('Attention! Date en format str à convertir en format date')

df["Date d'arrivée\nArrival date"] = df["Unnamed: 7"]
df.loc[1:,"Écarts\nGaps"] = df.loc[1:,"Unnamed: 13"]
df.loc[1:,"Unnamed: 14"] = df.loc[1:,"Unnamed: 15"]
df = df.drop(["Unnamed: 5","Unnamed: 6","Unnamed: 7", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 13", "Unnamed: 15"], 1)
df.head()"""
df

In [ ]:
RankList

In [ ]:
from urllib import request
import shutil
url = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_140000.xlsx"
filename = "vendeeglobe_20201108_140000.xlsx"

with request.urlopen(url) as response, open(filename, 'wb') as out_file: shutil.copyfileobj(response, out_file)

In [ ]:
df = pd.read_excel ("vendeeglobe_20201108_140000.xlsx",sheet_name=0, header=3)
df